In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1.describe(include='object')

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769
unique,7518,4243,128,177,449,2358,343
top,4675,770,117961,118300,117878,117906,118322
freq,839,152,21407,4424,1135,6896,4649


In [6]:
x=df1.drop('target',axis=1)
y=df1['target']

In [7]:
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,StratifiedKFold,KFold

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=123)
te=TargetEncoder(return_df=True)
xtr_enc=te.fit_transform(x_train,y_train)
xts_enc=te.transform(x_test)

In [9]:
xts_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9831 entries, 5666 to 12899
Data columns (total 7 columns):
RESOURCE            9831 non-null float64
MGR_ID              9831 non-null float64
ROLE_ROLLUP_1       9831 non-null float64
ROLE_ROLLUP_2       9831 non-null float64
ROLE_DEPTNAME       9831 non-null float64
ROLE_FAMILY_DESC    9831 non-null float64
ROLE_CODE           9831 non-null float64
dtypes: float64(7)
memory usage: 614.4 KB


In [10]:
y_test.value_counts()

1    9260
0     571
Name: target, dtype: int64

# Target Encoding

In [11]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score,roc_curve,accuracy_score,confusion_matrix,classification_report

In [12]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    print('AUC_ROC= ',auc)
    print('Accuracy= ',acc)
    print('confusion matrix:\n',confusion_matrix(y,pred))
    

In [13]:
lgb=LGBMClassifier()
params={'num_leaves': st.randint(10,50),
    'max_depth':st.randint(3,30) ,
    'learning_rate':st.uniform(0,1) ,
    'n_estimators':st.randint(100,250)}
sf=StratifiedKFold()
rs=RandomizedSearchCV(lgb,param_distributions=params,cv=sf,scoring='roc_auc',random_state=1234)
m=rs.fit(xtr_enc,y_train)
lbp=m.best_params_
print('RS:',m.best_score_)
print('params: ',lbp)
lgb=LGBMClassifier()
mod=lgb.fit(xtr_enc,y_train)
print('\nTrain:')
aucroc(mod,xtr_enc,y_train)
print('\nTest:')
aucroc(mod,xts_enc,y_test)


RS: 0.981487775640295
params:  {'learning_rate': 0.022143913603027254, 'max_depth': 9, 'n_estimators': 239, 'num_leaves': 10}

Train:
AUC_ROC=  0.9950184614770464
Accuracy=  0.9805126863719592
confusion matrix:
 [[ 1040   286]
 [  161 21451]]

Test:
AUC_ROC=  0.7922589485310527
Accuracy=  0.9407995117485505
confusion matrix:
 [[ 158  413]
 [ 169 9091]]


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [20]:
lr=LogisticRegression()
params={}
sf=StratifiedKFold()
rs=RandomizedSearchCV(lr,param_distributions=params,cv=sf,scoring='roc_auc',random_state=1234)
m=rs.fit(xtr_enc,y_train)
lbp=m.best_params_
print('RS:',m.best_score_)
print('params: ',lbp)
lgb=LGBMClassifier()
mod=lr.fit(xtr_enc,y_train)
print('\nTrain:')
aucroc(mod,xtr_enc,y_train)
print('\nTest:')
aucroc(mod,xts_enc,y_test)


C:\Users\Shashi Katteri\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RS: 0.962586353531084
params:  {}

Train:
AUC_ROC=  0.9627157793740084
Accuracy=  0.9608945854041329
confusion matrix:
 [[  624   702]
 [  195 21417]]

Test:
AUC_ROC=  0.8316655823401027
Accuracy=  0.9436476452039467
confusion matrix:
 [[ 127  444]
 [ 110 9150]]


In [21]:
rf=RandomForestClassifier()
params={
    'max_depth':st.randint(3,30) ,

    'n_estimators':st.randint(100,250)}
sf=StratifiedKFold()
rs=RandomizedSearchCV(rf,param_distributions=params,cv=sf,scoring='roc_auc',random_state=1234)
m=rs.fit(xtr_enc,y_train)
lbp=m.best_params_
print('RS:',m.best_score_)
print('params: ',lbp)
lgb=LGBMClassifier()
mod=rf.fit(xtr_enc,y_train)
print('\nTrain:')
aucroc(mod,xtr_enc,y_train)
print('\nTest:')
aucroc(mod,xts_enc,y_test)


RS: 0.97784614450054
params:  {'max_depth': 12, 'n_estimators': 130}

Train:
AUC_ROC=  0.9995821863391351
Accuracy=  0.9961635713662917
confusion matrix:
 [[ 1263    63]
 [   25 21587]]

Test:
AUC_ROC=  0.7849834324987801
Accuracy=  0.9434442070999898
confusion matrix:
 [[ 181  390]
 [ 166 9094]]
